# Server.py 
## Federated Learning Server
Accepts connection from a set number of clients. Has access to a repository of unlabeled public data. Once all clients send predictions on public data, server aggregates results. Once resulrs are compiled the server sends out the public dataset to supliment client training with the newly assigned labels.

In [ ]:
import send_receive
import socket
import threading
import torchvision.datasets as datasets
import numpy as np

In [ ]:
def load_dataset():
    mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
    trainimages = mnist_trainset.data
    return trainimages

In [ ]:
HOST = "0.0.0.0"
PORT = 3505

NUM_CLIENTS = 10
NUM_ROUNDS = 10
NUM_CLASSES = 10

logit_dict = {}
num_responses = 0
agreggation_done = 0

public_data = load_dataset()
labels = np.empty(len(public_data))

In [ ]:
def aggregation_rule(logits_dict):
    #TODO: Define aggregation [logits(CLIENTS, IMAGES, CLASSES)->in, labels(IMAGES)->out]
    if num_responses >= NUM_CLIENTS:
        aggregated_logits = np.empty_like(labels)
        labels = aggregated_logits
        agreggation_done = 1
        num_responses = 0


In [ ]:
def handle_client(conn, addr, public_data):
    print(f"[+] Connected: {addr}")

    try:
        # INITIAL ROUND
        # 1. Send initial public data
        data = send_receive.send_numpy(public_data.to_numpy())
        conn.sendall(len(data).to_bytes(8, 'big'))
        conn.sendall(data)

        # 2. recieve public data logits
        size = int.from_bytes(conn.recv(8), 'big')
        payload = conn.recv(size)
        logits = send_receive.recv_numpy(payload)
        if not data:
            print(f"[-] {addr} disconnected before responding")
            conn.close()
            return
        
        #TODO: Aggregate
        #Then update global var logit
        num_repsonses += 1
        logit_dict[threading.get_ident()] = logits

        while agreggation_done != 1:
            continue
    
        agreggation_done = 0
        
        # SUBSEQUENT ROUNDS
        for i in range(NUM_ROUNDS):
            data = send_receive.send_numpy(labels.to_numpy())
            conn.sendall(len(data).to_bytes(8, 'big'))
            conn.sendall(data)

            size = int.from_bytes(conn.recv(8), 'big')
            payload = conn.recv(size)
            logits = send_receive.recv_numpy(payload)

            num_responses += 1
            logit_dict[threading.get_ident()] = logits

            while agreggation_done != 1:
                continue



    except ConnectionResetError:
        print(f"[-] Connection reset by {addr}")
    finally:
        conn.close()
        print(f"[-] Disconnected: {addr}")

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen()

    print(f"[SERVER] Listening on {HOST}:{PORT}")



    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr, public_data), daemon=True)
        aggregator = threading.Thread(target=aggregation_rule, args=(logit_dict), daemon=True)
        thread.start()
        aggregator.start()


In [34]:
start_server()

torch.Size([10, 60000, 10])
[SERVER] Listening on 0.0.0.0:3505


KeyboardInterrupt: 